In [1]:
import asyncio
import drawSvg as draw
import ipywidgets as widgets
import math
from IPython.display import display
from pynoise.noisemodule import Simplex
import pythonosc


noise = Simplex(seed=111)

width = 300
height = 300

d = draw.Drawing(width, height, origin='center', displayInline=True)
panel = {
    'seed': widgets.IntSlider(
        min=-10000,
        max=10000,
        step=1,
        description='seed',
        orientation='vertical',
        value=0
    ),
    'counter': widgets.IntSlider(
        min=0,
        max=100,
        step=1,
        description='counter',
        orientation='vertical',
        value=3
    ),
    'scale': widgets.FloatSlider(
        min=1,
        max=30,
        step=.2,
        description='scale',
        orientation='vertical',
        value=10
    ),
    'multiplier': widgets.FloatSlider(
        min=-50,
        max=50,
        step=.1,
        description='multiplier',
        orientation='vertical',
        value=0
    ),
    'angle': widgets.FloatSlider(
        min=-math.tau,
        max=math.tau,
        step=.01,
        description='angle',
        orientation='vertical',
        value=0
    ),
    'x': widgets.FloatSlider(
        min=-1000,
        max=1000,
        step=.5,
        description='x',
        orientation='vertical',
        value=0
    ),
    'y': widgets.FloatSlider(
        min=-1000,
        max=1000,
        step=.5,
        description='y',
        orientation='vertical',
        value=0
    ),
}
control_layout = widgets.Layout(
    overflow='scroll hidden',
    border='3px solid black',
    width='500px',
    height='',
    flex_flow='row',
    display='flex'
)
sketch = widgets.Output()

def fetch(key):
    return panel.get(key).value

def place(key, value):
    panel.get(key).value = value
    
def set_seed(seed):
    noise = Simplex(seed=seed)
    
def seed_listener(change):
    set_seed(change['new'])

def equi_tri(midX, midY, r, angle):
    p = draw.Path(stroke_width=2, stroke='lime', fill='none')
    x1 = midX + r * math.cos(angle);
    y1 = midY - r * math.sin(angle);
    x2 = midX + r * math.cos(angle + (math.tau/3));
    y2 = midY - r * math.sin(angle + (math.tau/3));
    x3 = midX + r * math.cos(angle + (2*math.tau/3));
    y3 = midY - r * math.sin(angle + (2*math.tau/3));
#     if (hasRays) {
#         line(midX, midY, x1, y1);
#         line(midX, midY, x2, y2);
#         line(midX, midY, x3, y3);
#     }

    p.M(x1, y1)  # Start path at point (-10, 20)
    p.L(x2, y2)
    p.L(x3, y3)
    p.L(x1, y1)
    return p


def print_volume_handler(unused_addr, args, volume):
  print("[{0}] ~ {1}".format(args[0], volume))

osc_port = 42069
dispatcher = pythonosc.dispatcher.Dispatcher()
dispatcher.map("/button1", print)
dispatcher.map("/radar1", print)
dispatcher.map("/f1", print)
dispatcher.map("/radial1", print)
dispatcher.map("/encoder1", print)
dispatcher.map('/xy1', print)

def dispatch(dispatch_map, dispatcher):
    dispatcher

async def start_osc_server():
    server = osc_server.AsyncIOOSCUDPServer(('0.0.0.0', osc_port), dispatcher, asyncio.get_event_loop())
    transport, protocol = await pythonosc.server.create_serve_endpoint()
    print("Serving on {}".format(server.server_address))

# start_server()
server = osc_server.BlockingOSCUDPServer(('0.0.0.0', osc_port), dispatcher)
server.serve_forever()  # Blocks forever

/radar1 0.2525339126586914 0.8217853903770447
/radar1 0.2525339126586914 0.7974482178688049
/radar1 0.20602010190486908 0.7974482178688049
/radar1 0.20602010190486908 0.7879348993301392
/radar1 0.1550583392381668 0.7879348993301392
/radar1 0.1550583392381668 0.76237952709198
/radar1 0.11413538455963135 0.76237952709198
/radar1 0.11413538455963135 0.7214470505714417
/radar1 0.08950798958539963 0.7214470505714417
/radar1 0.08950798958539963 0.6640135645866394
/radar1 0.08465004712343216 0.6640135645866394
/radar1 0.08465004712343216 0.6029971837997437
/radar1 0.08825314790010452 0.6029971837997437
/radar1 0.08825314790010452 0.554995059967041
/radar1 0.10379386693239212 0.554995059967041
/radar1 0.10379386693239212 0.5147066116333008
/radar1 0.11396755278110504 0.5147066116333008
/radar1 0.11396755278110504 0.49186164140701294
/radar1 0.13131113350391388 0.49186164140701294
/radar1 0.13131113350391388 0.4686342477798462
/radar1 0.14512401819229126 0.4686342477798462
/radar1 0.14512401819

In [7]:
def refresh(change):
    # Draw an irregular polygon
    # d.append(
    #     draw.Lines(
    #         0, 0, 0, width, height, width,height, 0,
    #         id='lines'
    #     )
    # )
    d = draw.Drawing(width, height, origin='center', displayInline=False)
    d.append(
        draw.Rectangle(
            -width/2, -height/2, width, height,
            id="border", stroke='cyan', fill='none'
        )
    )

    angle = fetch('angle')
    for c in range(fetch('counter')):
        angle = c * (math.degrees(math.tau) / fetch('counter'));
        r = fetch('y')
        m = fetch('scale')
        if (c % 2 == 0):
            r = fetch('x')
            m = fetch('multiplier')
        d.append(
            draw.Circle(0, r, m * 2,
                    id=f'circle-{c}', stroke_width=2, fill='none', stroke='pink', transform=f'rotate({angle})')
            )
#         d.append(equi_tri(fetch('x'), fetch('y'), (c+1)*fetch('scale'), angle))
#         angle += fetch('multiplier') * noise.get_value(c, angle, 99)
        angle += fetch('multiplier') * .01
        if angle > math.tau:
            angle -= math.tau
#         print(angle)
    d.append(draw.Circle(0, 0, max(fetch('x'), fetch('y')) + max(fetch('scale'), fetch('multiplier')) + 10, id='outline',stroke_width=2, fill='none', stroke='green'))
    # Draw an arbitrary path (a triangle in this case)

    d.setPixelScale(1)
    sketch.clear_output()
    with sketch:
        display(d)
        d.saveSvg('clayrings2.svg')

for control in panel.values():
    control.unobserve_all()
    control.observe(refresh, names='value')
    if control.description == 'seed':
        control.observe(seed_listener, names='value')

In [8]:
carousel = widgets.Box(children=[p for p in panel.values()], layout=control_layout)

print(noise.get_value(100,100,100))
display(widgets.VBox([widgets.Label('Scroll horizontally:'), carousel]))
display(sketch)
refresh(None)

0.0


Output()